# 实践任务：
![](https://ai-studio-static-online.cdn.bcebos.com/ed6691c01b5141b1940645c613c15045fb20ae2643164862b9bad94787e2cbf2)

# 数据集介绍
训练数据集为华南理工大学实验室公布的数据集

数据中包含500张女生图片，分别由70人进行打分，最终取平均值即为该图片的打分情况。

我们在实践中将图片分值设定为1-5。

500张图片中，450张用于训练，50张用于验证。

![](https://ai-studio-static-online.cdn.bcebos.com/9d213946134f4fc4abba86a4f5c8de829cf6b62b9d074fceb6d22c2e1c8fb71e)

In [7]:
# !unzip /home/aistudio/data/data18736/face_data_5.zip
# !pip install paddlepaddle==2.0.0
import paddle
paddle.fluid.install_check.run_check()

Running Verify Fluid Program ... 
Your Paddle Fluid works well on SINGLE GPU or CPU.
Your Paddle Fluid works well on MUTIPLE GPU or CPU.
Your Paddle Fluid is installed successfully! Let's start deep Learning with Paddle Fluid now


In [8]:
#导入必要的包
import os
import paddle
import numpy as np
from PIL import Image
import zipfile
import random
import json
import sys
import paddle.nn.functional as F
from paddle import ParamAttr
print("本教程基于Paddle的版本号为："+paddle.__version__)

本教程基于Paddle的版本号为：2.0.2


In [10]:
'''
参数配置
'''
train_parameters = {
    "input_size": [3, 100, 100],                              #输入图片的shape
    "class_dim": 5,                                          #分类数
    "src_path":"/home/aistudio/data/data18736/face_data_5.zip",#原始数据集路径
    "target_path":"/home/aistudio/face_data_5",                     #要解压的路径
    "train_list_path": "/home/aistudio/data/train.txt",       #train.txt路径
    "eval_list_path": "/home/aistudio/data/eval.txt",         #eval.txt路径
    "readme_path": "/home/aistudio/data/readme.json",         #readme.json路径
    "label_dict":{},                                          #标签字典
    "num_epochs": 40,                                         #训练轮数
    "train_batch_size": 16,                                    #训练时每个批次的大小
    "learning_strategy": {                                    #优化函数相关的配置
        "lr": 0.001                                           #超参数学习率
    } 
}

In [11]:
#解压数据集
src_path=train_parameters["src_path"]
target_path=train_parameters["target_path"]
if(not os.path.isdir(target_path)):
    z = zipfile.ZipFile(src_path, 'r')
    z.extractall(path=target_path)
    z.close()

# step1.数据准备


1、定义数据集类

2、定义train_dataset、test_dataset

In [12]:
class MyDataset(paddle.io.Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self, mode='train'):
        """
        步骤二：实现构造函数，定义数据集大小
        """
        super(MyDataset, self).__init__()
        self.data = []
        self.label = []
        data_path = ''
        train_list_path = '/home/aistudio/face_data_5/face_data_5/face_image_train'
        eval_list_path = '/home/aistudio/face_data_5/face_data_5/face_image_test'
        if mode == 'train':
            for image in os.listdir(train_list_path):
                label = int(image.split('-')[0]) - 1
                img_path = os.path.join(train_list_path+ '/' + image)
                img = Image.open(img_path)
                if img.mode != 'RGB': 
                        img = img.convert('RGB') 
                img = img.resize((100, 100), Image.BILINEAR)
                img = np.array(img).astype('float32') 
                img = img.transpose((2, 0, 1))  # HWC to CHW 
                img = img/255               # 像素值归一化 
                self.data.append(img)
                self.label.append(int(label))
                    
        else:
            for image in os.listdir(eval_list_path):
                label = int(image.split('-')[0]) - 1
                img_path = os.path.join(eval_list_path+ '/' + image)
                img = Image.open(img_path)
                if img.mode != 'RGB': 
                        img = img.convert('RGB') 
                img = img.resize((100, 100), Image.BILINEAR)
                img = np.array(img).astype('float32') 
                img = img.transpose((2, 0, 1))  # HWC to CHW 
                img = img/255               # 像素值归一化 
                self.data.append(img)
                self.label.append(int(label))
            
    def __getitem__(self, index):
        """
        步骤三：实现__getitem__方法，定义指定index时如何获取数据，并返回单条数据（训练数据，对应的标签）
        """
        #返回单一数据和标签
        data = self.data[index]
        label = self.label[index]
        #注：返回标签数据时必须是int64
        return data, np.array(label).astype('int64')
    def __len__(self):
        """
        步骤四：实现__len__方法，返回数据集总数目
        """
        #返回数据总数
        return len(self.data)

In [13]:
# 测试定义的数据集
train_dataset = MyDataset(mode='train')
eval_dataset = MyDataset(mode='val')
print('=============train_dataset =============')
#输出数据集的形状和标签
print(train_dataset.__getitem__(1)[0].shape,train_dataset.__getitem__(1)[1])
#输出数据集的长度
print(train_dataset.__len__())
print('=============eval_dataset =============')
#输出数据集的形状和标签
for data, label in eval_dataset:
    print(data.shape, label)
    break
#输出数据集的长度
print(eval_dataset.__len__())

=============train_dataset =============
(3, 100, 100) 2
450
=============eval_dataset =============
(3, 100, 100) 1
50


# step2、配置网络

搭建VGG网络

![](https://ai-studio-static-online.cdn.bcebos.com/0e54eee7249147f586d621523e6b9899abc3e8a42b6949faa6da268a8c214f52)

1.首先定义了一组卷积网络，即conv_block。卷积核大小为3x3，池化窗口大小为2x2，窗口滑动大小为2，groups决定每组VGG模块是几次连续的卷积操作，dropouts指定Dropout操作的概率。所使用的img_conv_group是在paddle.networks中预定义的模块，由若干组 Conv->BN->ReLu->Dropout 和一组 Pooling 组成。

2.五组卷积操作，即 5个conv_block。 第一、二组采用两次连续的卷积操作。第三、四、五组采用三次连续的卷积操作。每组最后一个卷积后面Dropout概率为0，即不使用Dropout操作。

3.最后接两层512维的全连接。

4.通过上面VGG网络提取高层特征，然后经过全连接层映射到类别维度大小的向量，再通过Softmax归一化得到每个类别的概率，也可称作分类器。


In [14]:
class ConvBnLayer(paddle.nn.Layer):
    """
    卷积 + batch_normal 层
    
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, act="relu"):
        super(ConvBnLayer, self).__init__()

        self._conv2d = paddle.nn.Conv2D(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding)

        self._batch_normal = paddle.nn.BatchNorm2D(
            num_features=out_channels,
            weight_attr=ParamAttr(),
            bias_attr=ParamAttr()
        )
        self.re = F.relu

    def forward(self, inputs):
        x = self._conv2d(inputs)
        x = self._batch_normal(x)
        x = self.re(x)
        return x

class ConvPool(paddle.nn.Layer):
    '''卷积+池化'''
    def __init__(self,
                 in_channels,
                 out_channels,
                 filter_size,
                 pool_size,
                 pool_stride,
                 groups,
                 pool_padding=0,
                 pool_type='max',
                 conv_stride=1,
                 conv_padding=0,
                 act='relu'):
        super(ConvPool, self).__init__()  
        self._conv2d_list = []
        for i in range(groups):
            conv2d = self.add_sublayer(   #返回一个由所有子层组成的列表。
                'bb_%d' % i,
                ConvBnLayer(
                    in_channels=in_channels, #通道数
                    out_channels=out_channels,   #卷积核个数
                    kernel_size=filter_size,   #卷积核大小
                    stride=conv_stride,        #步长
                    padding=conv_padding,      #padding大小，默认为0
                )
            )
        self._conv2d_list.append(conv2d) 
        if pool_type == 'avg':
            self._pool2d = paddle.nn.AvgPool2D(
                kernel_size=pool_size,           #池化核大小
                stride=pool_stride)        #池化步长
        elif pool_type == 'max':
            self._pool2d = paddle.nn.MaxPool2D(
                kernel_size=pool_size,           #池化核大小
                stride=pool_stride)        #池化步长
    def forward(self, inputs):
        x = inputs
        for conv in self._conv2d_list:
            x = conv(x)
        x = self._pool2d(x)
        return x

class VGGNet(paddle.nn.Layer):
    '''
    VGG网络
    '''
    def __init__(self):
        super(VGGNet, self).__init__()
        self.conv1 = ConvPool(3, 64, 3, 2, 2, 2)
        self.conv2  = ConvPool(64, 128, 3, 2, 2, 2)
        self.conv3  = ConvPool(128, 256, 3, 2, 2, 3)
        self.conv4  = ConvPool(256, 512, 3, 2, 2, 3)
        self.conv5  = ConvPool(512, 512, 3, 2, 2, 3)
        self.fc1 = paddle.nn.Linear(in_features=512, out_features=512)
        self.fc2 = paddle.nn.Linear(in_features=512, out_features=512)
        self.fc3 = paddle.nn.Linear(in_features=512, out_features=5)
       

    def forward(self, inputs):
        """前向计算"""
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = paddle.reshape(x, [x.shape[0], -1])

        x= F.dropout(x, 0.5)
        
        x = self.fc1(x)
        x = F.relu(x)
        x= F.dropout(x, 0.5)
        x = self.fc2(x)
        x = F.relu(x)
        x= F.dropout(x, 0.5)
        x = self.fc3(x)
        y = F.softmax(x)

        return y

In [15]:
mynet = VGGNet()
inputs = paddle.randn(shape=[2,3,100,100])
print(mynet(inputs))

Tensor(shape=[2, 5], dtype=float32, place=CUDAPlace(0), stop_gradient=False,
       [[0.02822993, 0.57898909, 0.31654748, 0.05131700, 0.02491652],
        [0.00583847, 0.00020575, 0.09411111, 0.89959538, 0.00024928]])


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:648: UserWarning: When training, we now always track global mean and variance.
  


# **三、模型训练 && 四、模型评估**

In [16]:
'''
模型训练
'''
print('model')
model = paddle.Model(VGGNet())
# 定义损失函数
print('prepare')
model.prepare(paddle.optimizer.Adam(parameters=model.parameters()),paddle.nn.CrossEntropyLoss(),paddle.metric.Accuracy())
# 训练可视化VisualDL工具的回调函数
visualdl = paddle.callbacks.VisualDL(log_dir='visualdl_log')
# 启动模型全流程训练
print('fit')
model.fit(train_dataset,            # 训练数据集
          eval_dataset,            # 评估数据集
          epochs=100,            # 总的训练轮次
          batch_size = 64,    # 批次计算的样本量大小
          shuffle=True,             # 是否打乱样本集
          verbose=1,                # 日志展示格式
          save_dir='./chk_points/', # 分阶段的训练模型存储路径
          callbacks=[visualdl])     # 回调函数使用
#保存模型
model.save('model_save_dir')

model
prepare
fit
The loss value printed in the log is the current step, and the metric is the average value of previous step.
Epoch 1/100


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._places = loader.places
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 8/8 [==============================] - loss: 1.4048 - acc: 0.3933 - 76ms/step
save checkpoint at /home/aistudio/chk_points/0
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 1/1 [==============================] - loss: 1.4996 - acc: 0.4000 - 29ms/step
Eval samples: 50
Epoch 2/100
step 8/8 [==============================] - loss: 1.4048 - acc: 0.4356 - 45ms/step
save checkpoint at /home/aistudio/chk_points/1
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 1/1 [==============================] - loss: 1.5053 - acc: 0.4000 - 28ms/step
Eval samples: 50
Epoch 3/100
step 8/8 [==============================] - loss: 1.9048 - acc: 0.4378 - 43ms/step
save checkpoint at /home/aistudio/chk_points/2
Eval begin...
The loss value printed in the log is the current batch, and the metric is the average value of previous step.
step 1/1 [====

In [17]:
def load_image(img_path):
        #打开图片
        img = Image.open(img_path)
        if img.mode != 'RGB': 
                img = img.convert('RGB') 
        img = img.resize((100, 100), Image.BILINEAR)
        img = np.array(img).astype('float32') 
        img = img.transpose((2, 0, 1))  # HWC to CHW 
        img = img/255               # 像素值归一化 
        im = np.expand_dims(img, axis=0)
        # 保持和之前输入image维度一致
        print('im_shape的维度：',im.shape)
        return im

In [18]:
#读入测试图片并展示
import matplotlib.pyplot as plt
infer_path='data/data18737/1.jpg'
img = Image.open(infer_path)
plt.imshow(img)   
plt.show()    


label_list = ["1", "2", "3", "4", "5"]
#载入要预测的图片
infer_img = load_image(infer_path)
#将图片变为数组
infer_img=np.array(infer_img).astype('float32')
#进行预测
# print(infer_img.shape)
result = model.predict(np.expand_dims(infer_img, axis=0))
# 输出预测结果
print('results',result)
print("infer results: %s" % label_list[np.argmax(result)])

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
2021-12-17 12:05:34,346 - INFO - font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/co

<Figure size 640x480 with 1 Axes>

im_shape的维度： (1, 3, 100, 100)
Predict begin...
step 1/1 [==============================] - 8ms/step
Predict samples: 1
results [(array([[9.0154861e-23, 2.0027293e-16, 1.0000000e+00, 4.2544599e-22,
        6.8037853e-25]], dtype=float32),)]
infer results: 3


In [19]:
!paddlex --export_inference --model_dir=./ --save_dir=./inference_mode

[12-17 12:08:10 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --export_inference --model_dir=./ --save_dir=./inference_mode
[12-17 12:08:10 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.1. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` h